In [8]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [9]:
files = []
for i in range(1, 10):
    path = "./data/TEST_Kim-Study{i}.csv".format(i=i)
    temp = pd.read_csv(path)
    files.extend(temp[['Dialogue State']].values)
    
files = np.concatenate(files)
print(files)

['S2' 'S1' 'S3' ... 'S7' 'S1' 'S7']


In [10]:
IDEMap = {'D7': 'D6D15', 'D8': 'D6D15', 'E3': 'D10', 'E1': 'D11', 'E2': 'D13', 'D18': 'D6D15D17', 'D19': 'D1D2',
         'D21': 'D3D2', 'D22': 'D3D1', 'D23': 'D3D17', 'D25': 'D1D2D9', 'D27': 'DD3D17D1', 'D28': 'D5', 'D29': 'D17D14',
         'D30': 'D6', 'D31': 'D5D2', 'D32': 'D10', 'D33': 'D6', 'D34': 'D16D1', 'D36': 'D17D16', 'D37': 'D1D17',
         'D38': 'D3D17D1', 'D39': 'D3D6D5', 'D40': 'D17D1', 'D41': 'D7','D42': 'D13','D43': 'D13','D44': 'D13'}
temp = [int(i[1:]) for i in files]
data = np.array(temp)


In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [12]:
input_data, output_data = create_dataset(data, look_back=30)
tree_input = input_data
tree_output = output_data

In [13]:
input_data = np.reshape(input_data, (input_data.shape[0], len(input_data[0]), 1)).astype('int64')
output_data = output_data.astype('int64')
print(output_data)


[ 1  6 11 ... 15  7  1]


In [7]:
from keras import backend as K
from keras import metrics
from keras.optimizers import Adam
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = []
precision = []
recall = []


def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units=64, activation='relu', input_shape=(len(input_data[0]),1), dropout = 0.3, recurrent_dropout = 0.3),
        tf.keras.layers.Dense(units=128, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         y_train = y_train.reshape(-1)
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        
        print(f1_score(y_test, y_pred_classes, average='weighted'))
        precision.append(precision_score(y_test, y_pred_classes, average='weighted', zero_division=1))
        recall.append(recall_score(y_test, y_pred_classes, average='weighted', zero_division=1))

    return np.mean(scores)

# Load the input data and one-hot encode it
X = input_data
y = output_data
model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)
print("f1: ", np.mean(f1))
print("precision: ", np.mean(precision))
print("recall: ", np.mean(recall))

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 

22/22 - 0s - loss: 53.7165 - accuracy: 0.2085 - 417ms/epoch - 19ms/step
22/22 [==============================] - 0s 3ms/step
0.0
22/22 - 0s - loss: 47.1188 - accuracy: 0.1718 - 99ms/epoch - 4ms/step
22/22 [==============================] - 0s 4ms/step
0.0
22/22 - 0s - loss: 56.5921 - accuracy: 0.1659 - 100ms/epoch - 5ms/step
22/22 [==============================] - 0s 4ms/step
0.0
22/22 - 0s - loss: 44.6134 - accuracy: 0.1853 - 96ms/epoch - 4ms/step
22/22 [==============================] - 0s 3ms/step
0.0
22/22 - 0s - loss: 51.2997 - accuracy: 0.1853 - 123ms/epoch - 6ms/step
22/22 [==============================] - 0s 5ms/step
0.0
Cross-validated accuracy: 0.18336874842643738
f1:  nan
precision:  1.0
recall:  0.0


C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[0.0, 0.0, 0.0, 0.0, 0.0] [1.0, 1.0, 1.0, 1.0, 1.0] [0.0, 0.0, 0.0, 0.0, 0.0]


NameError: name 'autoencoder' is not defined

NameError: name 'autoencoder' is not defined

NameError: name 'autoencoder' is not defined

[[ 2  1  3 ...  6  1  6]
 [ 1  3  4 ...  1  6  1]
 [ 3  4  1 ...  6  1  6]
 ...
 [ 6 15 11 ...  6  6 35]
 [15 11  1 ...  6 35 15]
 [11  1 16 ... 35 15  7]]


In [22]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

# Input and output data
X = np.array(tree_input)
y = np.array(tree_output)
# Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
score = []
for train_idx, test_idx in kf.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    # Fit decision tree classifier
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    score.append(clf.score(X_test, y_test))
    # Predict test data and compute confusion matrix
    
print(np.mean(score))

0.25418372635397773
